In [51]:
import numpy as np
import cv2
import json
import os 

In [52]:
JSON_PATH = 'project-4-at-2023-12-12-17-27-7d370c76.json'
IMAGES_PATH = 'images5'
LABELS_PATH = 'labels5'
LABELS_COLOR_PATH = 'labels5_color'

# TODO buat list di text aja
SWAN = 1
DUCK = 2

SWAN_COLOR = (255,0,0)
DUCK_COLOR = (0,255,0)



if not os.path.exists(LABELS_PATH): 
    os.makedirs(LABELS_PATH) 

if not os.path.exists(LABELS_COLOR_PATH): 
    os.makedirs(LABELS_COLOR_PATH) 

In [53]:
class InputStream:
    def __init__(self, data):
        self.data = data
        self.i = 0

    def read(self, size):
        out = self.data[self.i : self.i + size]
        self.i += size
        return int(out, 2)


def access_bit(data, num):
    """from bytes array to bits by num position"""
    base = int(num // 8)
    shift = 7 - int(num % 8)
    return (data[base] & (1 << shift)) >> shift


def bytes2bit(data):
    """get bit string from bytes data"""
    return ''.join([str(access_bit(data, i)) for i in range(len(data) * 8)])


def decode_rle(rle, print_params: bool = False):
    """from LS RLE to numpy uint8 3d image [width, height, channel]

    Args:
        print_params (bool, optional): If true, a RLE parameters print statement is suppressed
    """
    input = InputStream(bytes2bit(rle))
    num = input.read(32)
    word_size = input.read(5) + 1
    rle_sizes = [input.read(4) + 1 for _ in range(4)]

    if print_params:
        print(
            'RLE params:', num, 'values', word_size, 'word_size', rle_sizes, 'rle_sizes'
        )

    i = 0
    out = np.zeros(num, dtype=np.uint8)
    while i < num:
        x = input.read(1)
        j = i + 1 + input.read(rle_sizes[input.read(2)])
        if x:
            val = input.read(word_size)
            out[i:j] = val
            i = j
        else:
            while i < j:
                val = input.read(word_size)
                out[i] = val
                i += 1
    return out


In [55]:
try:
    f = open(JSON_PATH)
    data = json.load(f)
    for dt in data:
        file_upload = dt['file_upload']
        # print(file_upload)
        img = cv2.imread(IMAGES_PATH + '\\' + file_upload)
        # print(file_upload)
        if img is None or img.shape[0] == 0 or img.shape[1] == 0:
            # print("Error: Gambar tidak dapat dibaca atau tidak memiliki bentuk yang valid.")
            continue

        height, width, channels = img.shape
        label_image = np.zeros([height, width], dtype=np.uint8)
        label_color = np.zeros([height, width, 3], dtype=np.uint8)

        # Proses anotasi gambar dan penyimpanan label di sini
        for ano in dt['annotations']:
            for res in ano['result']:
                ori_height = res['original_height']
                ori_width = res['original_width']
                if height!=ori_height and width!=ori_width:
                    raise Exception("Size not match!")

                label = res['value']['brushlabels'][0]
                labelnum = 0
                labelcol = (255,255,255)
                if label=='Swan':
                    labelnum = SWAN
                    labelcol = SWAN_COLOR
                elif label=='Duck':
                    labelnum = DUCK
                    labelcol = DUCK_COLOR
                # print(label)
                
                rle = res['value']['rle']
                # lbl_rle = np.reshape(decode_rle(rle), [height, width, 4])[:, :, 3]
                lbl_rle = np.reshape(decode_rle(rle, print_params=True), [height, width, 4])[:, :, 3]
                label_num  = np.full([height, width], labelnum, dtype=np.uint8) 
                label_ano = cv2.bitwise_and(lbl_rle, label_num)
                label_image = np.where(label_ano == 0, label_image, label_ano)

                np.set_printoptions(threshold=np.inf)
                # print(label_image)
                # print(rle)
                # print((lbl_rle))
                

                lbl_rle_col = cv2.merge((lbl_rle,lbl_rle,lbl_rle))
                label_num_col  = np.full([height, width, 3], labelcol, dtype=np.uint8)
                label_ano_col = cv2.bitwise_and(lbl_rle_col, label_num_col)
                label_color = np.where(label_ano_col == [0,0,0], label_color, label_ano_col) # overlap
                
                
        cv2.imwrite(LABELS_PATH + '\\' + file_upload, label_image)
        cv2.imwrite(LABELS_COLOR_PATH + '\\' + file_upload, label_color)

except Exception as e:
    print("Error:", e)

RLE params: 1228800 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1811200 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 9216000 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 4000000 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 2775556 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 7052800 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 8294400 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 3686400 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 8294400 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1287680 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 3279360 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 9216000 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 3004000 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 20000000 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 2876400 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 7056000 values 8 word_size [3, 4, 8, 16] r